In [31]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from SALib.sample.morris import sample
from Core.inputparse import Parser
import os
from pathlib import Path

In [2]:
tmap = pd.ExcelFile('Data/Techmap/DEModel_V2.xlsx')
df = pd.read_excel(tmap,"ConversionSubProcess",skiprows=[1,2]).drop(columns=['is_storage', 'efficiency', 'technical_availability', 'output_profile', 'availability_profile'])

In [3]:
cs_names = np.array([])
cs_values = np.array([])
pattern = re.compile(r'\s*;\s*|\s+')
for i in range(len(df)):
    if df.iloc[i]["conversion_process_name"] == 'DEBUG':
        break
    else:
        if type(df.iloc[i]['conversion_process_name']) != str:
            continue
        cs = df.iloc[i].dropna()
        cs_name = f'{cs["conversion_process_name"]}&{cs["commodity_in"]}&{cs["commodity_out"]}'
        remaining_columns = cs.index
        for col in remaining_columns[4:]:
            value = cs[col]
            if type(value) == str:
                year_values = pattern.split(value[1:-1])
                for j in range(0, len(year_values), 2):
                    year_value = 0 if year_values[j+1] == 'NaN' else float(year_values[j+1])
                    cs_names = np.append(cs_names, f'{cs_name}&{col}&{year_values[j]}')
                    cs_values = np.append(cs_values, year_value)
            else:
                cs_names = np.append(cs_names, f'{cs_name}&{col}')
                cs_values = np.append(cs_values, value)

In [4]:
fixed_values = np.array([])
fixed_names = np.array([])
range_values = np.empty((0,2), dtype=float)
range_names = np.array([])

# TODO revise these definitions
fraction_params = ['efficiency_charge', 'out_frac_min', 'out_frac_max', 'in_frac_min', 'in_frac_max']
params_5_percent = ['efficiency_charge', 'c_rate', 'technical_lifetime']
params_20_percent = ['opex_cost_energy', 'capex_cost_power']
for i in range(len(cs_names)):
    if any(substring in cs_names[i] for substring in params_20_percent):
        percentage_range = 0.2
    elif any(substring in cs_names[i] for substring in params_5_percent):
        percentage_range = 0.05
    else:
        percentage_range = 0.1
    is_fraction = True if any(substring in cs_names[i] for substring in fraction_params) else False
        
    if cs_values[i] == 0 or cs_values[i] == 1:
        fixed_names = np.append(fixed_names, cs_names[i])
        fixed_values = np.append(fixed_values, cs_values[i])
    else:
        value_range = [cs_values[i]*(1-percentage_range), cs_values[i]*(1+percentage_range)]
        if is_fraction:
            if value_range[0] < 0:
                value_range -= value_range[0]
            elif value_range[1] > 1:
                value_range -= (value_range[1] - 1)
        range_names = np.append(range_names, cs_names[i])
        range_values = np.vstack((range_values, value_range))

In [5]:
resulting_inputs = {"fixed_names":fixed_names,"fixed_values":fixed_values,"range_names":range_names,"range_values":range_values}

In [19]:
with open('GSASamples/input_to_sample.pkl', 'wb') as f:
    pickle.dump(resulting_inputs, f, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
# Define the model inputs
# TODO set output names
problem = {
    'num_vars': len(range_names),
    'names': range_names.tolist(),
    'bounds': range_values.tolist()
}

In [8]:
# Generate samples
trajectories = 100
optimal_trajectories = 4
start_time = time.time()
param_values = sample(problem, trajectories, optimal_trajectories=optimal_trajectories)
sampling_time = time.time() - start_time

In [9]:
run_params = {"trajectories":trajectories,"optimal_trajectories":optimal_trajectories,"run_time":sampling_time}
samples_result = {"run_params":run_params,"var_names":range_names,"samples":param_values}

In [20]:
with open('GSASamples/samples_morris.pkl', 'wb') as f:
    pickle.dump(samples_result, f, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
objects = []
with (open("samples_morris.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [12]:
samples = objects[0]['samples']
var_names = objects[0]['var_names']

In [13]:
tmap = pd.ExcelFile('Data/Techmap/DEModel_V2.xlsx')
base_cs = pd.read_excel(tmap,"ConversionSubProcess",  skiprows=[1, 2])

In [14]:
input_dfs = []

for i in range(len(samples)):
    mod_cs = base_cs.copy()
    last_yearly = ''
    yearly_input = []
    for j in range(len(var_names)):
        split_name = var_names[j].split('&')
        is_yearly = False if len(split_name) == 4 else True
        if is_yearly:
            if var_names[j][:-5] != last_yearly:
                last_yearly = var_names[j][:-5]
                yearly_input = f'[{split_name[-1]} {samples[i][j]}'
            else:
                yearly_input = f'{yearly_input}; {split_name[-1]} {samples[i][j]}'
                if j != len(var_names) - 1:
                    if var_names[j+1][:-5] != var_names[j][:-5]:
                        yearly_input = f'{yearly_input}]'
                        mod_cs.loc[(mod_cs['conversion_process_name']==split_name[0])&
                                   (mod_cs['commodity_in']==split_name[1])&
                                   (mod_cs['commodity_out']==split_name[2]),split_name[3]] = yearly_input
        else:
            mod_cs.loc[(mod_cs['conversion_process_name']==split_name[0])&
                       (mod_cs['commodity_in']==split_name[1])&
                       (mod_cs['commodity_out']==split_name[2]),split_name[3]] = samples[i][j]
    input_dfs.append(mod_cs)

In [15]:
model_name = "DEModel_V2"
scenario = 'Base'
ts_path = Path(".").joinpath(os.getcwd(), 'Data', 'TimeSeries')
techmap_path = Path(".").joinpath(os.getcwd(), 'Data', 'Techmap')
parser = Parser(model_name, techmap_dir_path=techmap_path, ts_dir_path=ts_path ,scenario = scenario) 

In [ ]:
parsed_samples = []
parser = Parser(model_name, techmap_path, ts_path, scenario)
for idf in input_dfs:
    parser.parse(idf)
    parsed_samples.append(parser.get_input())

In [17]:
len(parsed_samples)

796

In [21]:
with open('GSASamples/parsed_samples_morris.pkl', 'wb') as f:
    pickle.dump(parsed_samples, f, protocol=pickle.HIGHEST_PROTOCOL)

In [79]:
objects = []
with (open("GSAResults/results_simulations.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [82]:
var_names = objects[0]['output_var_names']
with open('GSAResults/var_names.pkl', 'wb') as f:
    pickle.dump(var_names, f, protocol=pickle.HIGHEST_PROTOCOL)

In [2]:
test = []
with (open("GSAResults/DEModel_V2-Base-16-02-24_12-02/results_simulations.pkl", "rb")) as openfile:
    while True:
        try:
            test.append(pickle.load(openfile))
        except EOFError:
            break

In [3]:
import numpy as np
np.sum(test[0]['infeasible'])

796

In [1]:
import click
import os
import time 
import pickle
from pathlib import Path
from datetime import datetime

from InquirerPy import prompt

# -- internal imports -- #
from Core.inputparse import Parser
from Core.model import Model
from Core.datacls import save_input_output, read_input_output
from Core.plotter import Plotter, PlotType
from Core.gsasampler import GSASampler

In [2]:
TECHMAP_DIR_PATH = Path(".").joinpath(os.getcwd(), 'Data', 'Techmap')
TS_DIR_PATH = Path(".").joinpath(os.getcwd(), 'Data', 'TimeSeries')
RUNS_DIR_PATH = Path(".").joinpath(os.getcwd(), 'Runs')
GSA_DIR_PATH = Path(".").joinpath(os.getcwd(), 'GSAResults')
FNAME_MODEL = 'input_output.pkl'

In [9]:
parser = Parser('DEModel_V2', techmap_dir_path=TECHMAP_DIR_PATH, ts_dir_path=TS_DIR_PATH ,scenario = 'Base') 

parser.parse()
model_input = parser.get_input()

In [10]:
model_input

Input(param=Param(globalparam=GlobalParam(dt=3.0, discount_rate=0.05, w=13.035714285714285, discount_factor={Year(_value=2015): 1.0, Year(_value=2020): 0.7835261664684589, Year(_value=2025): 0.6139132535407591, Year(_value=2030): 0.48101709809097004, Year(_value=2035): 0.3768894828730004, Year(_value=2040): 0.29530277169776187, Year(_value=2045): 0.23137744865585788, Year(_value=2050): 0.181290285352577, Year(_value=2055): 0.14204568230027764, Year(_value=2060): 0.11129650891613317}), cost=CostParam(opex_cost_energy=defaultdict(<function helper_zero at 0x7f723a7f3d90>, {((ConversionProcess(_value='Import_Biomass'),Commodity(_value='Dummy'),Commodity(_value='Biomass')), Year(_value=2015)): 0.02, ((ConversionProcess(_value='Import_Biomass'),Commodity(_value='Dummy'),Commodity(_value='Biomass')), Year(_value=2020)): 0.02, ((ConversionProcess(_value='Import_Biomass'),Commodity(_value='Dummy'),Commodity(_value='Biomass')), Year(_value=2025)): 0.02, ((ConversionProcess(_value='Import_Biomass

In [3]:
sampler = GSASampler('DEModel_V2', techmap_dir_path=TECHMAP_DIR_PATH, ts_dir_path=TS_DIR_PATH ,scenario = 'Base')
input_samples, samples_df = sampler.generate_morris_samples('1')

In [6]:
test = samples_df[0]

In [4]:
sample_input = []
with (open(f'GSASamples/morris_298-samples_9s_24-02-24_15-46.pkl', "rb")) as openfile:
    while True:
        try:
            sample_input.append(pickle.load(openfile))
        except EOFError:
            break
sample_without = sample_input[0][0]

sample_input = []
with (open(f'GSASamples/morris_398-samples_21s_24-02-24_15-48.pkl', "rb")) as openfile:
    while True:
        try:
            sample_input.append(pickle.load(openfile))
        except EOFError:
            break
sample_with = sample_input[0][0]
    

In [5]:
input_samples

array(['Import_Biomass&Dummy&Biomass&opex_cost_energy',
       'Import_Coal&Dummy&Coal&opex_cost_energy&2016',
       'Import_Coal&Dummy&Coal&opex_cost_energy&2030',
       'Import_Coal&Dummy&Coal&opex_cost_energy&2050',
       'Import_Crude_Oil&Dummy&Crude_Oil&opex_cost_energy&2016',
       'Import_Crude_Oil&Dummy&Crude_Oil&opex_cost_energy&2030',
       'Import_Crude_Oil&Dummy&Crude_Oil&opex_cost_energy&2050',
       'Import_Lignite&Dummy&Lignite&opex_cost_energy',
       'Import_Natural_Gas&Dummy&Natural_Gas&opex_cost_energy&2016',
       'Import_Natural_Gas&Dummy&Natural_Gas&opex_cost_energy&2030',
       'Import_Natural_Gas&Dummy&Natural_Gas&opex_cost_energy&2050',
       'Import_GreenH2&Dummy&H2&opex_cost_energy&2020',
       'Import_GreenH2&Dummy&H2&opex_cost_energy&2030',
       'Import_GreenH2&Dummy&H2&opex_cost_energy&2050',
       'Import_GreenH2&Dummy&H2&opex_cost_energy&2070',
       'PP_Biomass&Biomass&Electricity&capex_cost_power&2020',
       'PP_Biomass&Biomass&Electri

In [3]:
# Build
model_input = input_samples[0]
model_instance = Model(model_input)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2466960
Academic license 2466960 - for non-commercial use only - registered to gu___@stud.tu-darmstadt.de


In [39]:
input_samples[0]

Input(param=Param(globalparam=GlobalParam(dt=3.0, discount_rate=0.05, w=13.035714285714285, discount_factor={Year(_value=2015): 1.0, Year(_value=2020): 0.7835261664684589, Year(_value=2025): 0.6139132535407591, Year(_value=2030): 0.48101709809097004, Year(_value=2035): 0.3768894828730004, Year(_value=2040): 0.29530277169776187, Year(_value=2045): 0.23137744865585788, Year(_value=2050): 0.181290285352577, Year(_value=2055): 0.14204568230027764, Year(_value=2060): 0.11129650891613317}), cost=CostParam(opex_cost_energy=defaultdict(<function helper_zero at 0x7fe0a6ca7d90>, {((ConversionProcess(_value='Import_Biomass'),Commodity(_value='Dummy'),Commodity(_value='Biomass')), Year(_value=2015)): 0.018666666666666668, ((ConversionProcess(_value='Import_Biomass'),Commodity(_value='Dummy'),Commodity(_value='Biomass')), Year(_value=2020)): 0.018666666666666668, ((ConversionProcess(_value='Import_Biomass'),Commodity(_value='Dummy'),Commodity(_value='Biomass')), Year(_value=2025)): 0.01866666666666

In [7]:
model = model_instance.getModel()

In [8]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.1 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2466960 - for non-commercial use only - registered to gu___@stud.tu-darmstadt.de
Optimize a model with 1048249 rows, 922853 columns and 2858223 nonzeros
Model fingerprint: 0xf6aa7920
Coefficient statistics:
  Matrix range     [6e-06, 2e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 7e+05]
Presolve removed 101609 rows and 248489 columns
Presolve time: 0.20s

Solved in 0 iterations and 0.20 seconds (0.10 work units)
Infeasible model


In [38]:
ubpen = [1.0]*model.numVars
vars = model.getVars()
model.feasRelax(1, False, vars, None, ubpen, None, None)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.1 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads



0.0

In [25]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.1 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2466960 - for non-commercial use only - registered to gu___@stud.tu-darmstadt.de
Optimize a model with 1971102 rows, 1845706 columns and 4703929 nonzeros
Model fingerprint: 0x24801afd
Coefficient statistics:
  Matrix range     [6e-06, 2e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 7e+05]
Presolve removed 922853 rows and 1089353 columns
Presolve time: 0.29s

Solved in 0 iterations and 0.29 seconds (0.20 work units)
Infeasible model
